### 优点：在数据较少的情况下仍然有效，可以处理多类别问题 
### 缺点：对于输入数据的准备方式较为敏感。
### 适用数据类型：标称型数据


####  朴素贝叶斯一般过程

#### 1.收集数据：可以使用任何方法
#### 2.准备数据：需要数值型或者布尔型数据
#### 3.有大量特征时，绘制特征作用不大，此时使用直方图效果更好
#### 4.训练算法：计算不同独立特征的条件概率
#### 5.测试算法：计算错误率
#### 6.使用算法：一个常见的朴素贝叶斯应用是文档分类。可以在任意的分类场景中使用朴素贝叶斯分类器，不一定非要是文本

In [31]:
from numpy import *

###  一 准备数据：从文本中构建词向量

In [32]:
#创建实验样本 该函数返回的第一个变量是进行词条切分后的文档集合
#返回的第二个变量是一个类别标签的集合。这里有两类，侮辱性和非侮辱性
def loadDataSet():
    postingList = [['my','dog','has','flea','problem','help','please'],\
                   ['maybe','not','take','him','to','dog','park','stupid'],\
                   ['my','dalmation','is','so','cute','I','love','him'],\
                   ['stop','posting','stupid','worthless','garbage'],\
                   ['mr','licks','ate','my','steak','how','to','stop','him'],\
                   ['quit','buying','worthless','dog','food','stupid']]
    classVec = [0,1,0,1,0,1] #1代表侮辱性文字，0代表正常言论
    return postingList,classVec

In [33]:
#创建一个包含在所有文档中出现的不重复词的列表
def createVocabList(dataSet):
    #set() 函数创建一个无序不重复元素集，可进行关系测试，删除重复数据，还可以计算交集、差集、并集等
    vocabSet = set([])
    for document in dataSet:
        #两个set集合求并集
        vocabSet = vocabSet | set(document)
    #生成不重复的词汇列表
    return list(vocabSet)

In [34]:
#函数的输人参数为词汇表及某个文档 ，输出的是文档向量，向量的每一元素为1或0 ，分别表示词汇表中的单词在输人文档中是否出现
def setOfWords2Vec(vocabList,inputSet):
    #列表对 + 和 * 的操作符与字符串相似。+ 号用于组合列表，* 号用于重复列表。
    returnVec = [0]*len(vocabList)#初始化一个和词汇表相同大小的向量;
    for word in inputSet:#遍历文档中的单词
        if word in vocabList:#如果出现了词汇表中的单词，则将输出的文档向量（returnVec）中的对应值设为1
            returnVec[vocabList.index(word)] = 1
        else:print "the word:%s is not in my Vocabulary!" %word#否则打印不在列表中
    #返回文档向量
    return returnVec

In [35]:
#listOPosts, listClasses = loadDataSet()

In [36]:
#myVocabList = createVocabList(listOPosts)

### 二 训练算法：从词向量计算概率

In [37]:
#输入参数是文档矩阵trainMatrix，以及每篇文档类别标签所构成的向量。
def trainNB0(trainMatrix,trainCategory):
    #获取在测试文档矩阵中有几篇文档
    numTrainDocs = len(trainMatrix)
    #获取第一篇文档的单词长度
    numWords = len(trainMatrix[0])
    #类别为1的个数除以总篇数，就得到类别为1的文档在总文档数中所占的比例
    pAbusive = sum(trainCategory)/float(numTrainDocs)
  #  p0Num = ones(numWords);p1Num = ones(numWords)#初始化求概率的分子变量和分母变量，
    #p0Denom = 2.0;p1Denom = 2.0  #这里防止有一个p(xn|1)为0，则最后的乘积也为0，所有将分子初始化为1，分母初始化为2。
    p0Num = zeros(numWords);p1Num = zeros(numWords)
    p0Denom = 0.0;p1Denom = 0.0
    for i in range(numTrainDocs):#对每一篇训练文档
        if trainCategory[i] == 1:#如果这篇文档的类别是1
            # 对所有属于类别1的文档向量按位置累加，
            #最终形成的向量其实表示了在类别1下，一共包含了多少个词汇表中的不同单词（指列，一列表示一个单词，每一列都不重复）
            #每一列的值表示该列的单词出现了在属于类别1的所有文档中出现了几次
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])#sum函数计算对应文档向量的和,累加文档向量的和
        else:
            p0Num += trainMatrix[i]#
            p0Denom += sum(trainMatrix[i])
    #防止太多的很小的数相乘造成下溢。对乘积取对数。
    p1Vect =p1Num/p1Denom#change to log() 类别1
    p0Vect =p0Num/p0Denom #change to log()#返回每个类别的条件概率，不是常数，是向量，在向量里面是和词汇表向量长度相同，每个位置代表这个单词在这个类别中的概率
    return p0Vect,p1Vect,pAbusive #返回两个向量和一个概率

In [38]:
listOPosts, listClasses = loadDataSet()

In [39]:
myVocabList = createVocabList(listOPosts)

In [40]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))

In [41]:
p0V,p1V,pAb = trainNB0(trainMat, listClasses)

it's equal
